### Shapash Model Overview
https://shapash.readthedocs.io/en/latest/

##### With this tutorial you:
Understand how to create a Shapash SmartPredictor to make prediction and have local explanation in production with a simple use case.

This tutorial describes the different steps from training the model to Shapash SmartPredictor deployment. A more detailed tutorial allows you to know more about the SmartPredictor Object.

Contents:

- Build a Regressor
- Compile Shapash SmartExplainer
- From Shapash SmartExplainer to SmartPredictor
- Save Shapash Smartpredictor Object in pickle file
- Make a prediction

In [3]:
import seaborn as sns

In [4]:
df=sns.load_dataset('tips')

In [5]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [6]:
### Divide the dataset into independent and dependent dataset
y=df['tip']
X=df[df.columns.difference(['tip'])]

In [7]:
X.head()

,day,sex,size,smoker,time,total_bill
0,Sun,Female,2,No,Dinner,16.99
1,Sun,Male,3,No,Dinner,10.34
2,Sun,Male,3,No,Dinner,21.01
3,Sun,Male,2,No,Dinner,23.68
4,Sun,Female,4,No,Dinner,24.59


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.3 KB


In [9]:
X['day']=X['day'].cat.codes
X['sex']=X['sex'].cat.codes
X['smoker']=X['smoker'].cat.codes
X['time']=X['time'].cat.codes

/home/rakesh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/rakesh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/rakesh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

In [10]:
X

,day,sex,size,smoker,time,total_bill
0,3,1,2,1,1,16.99
1,3,0,3,1,1,10.34
2,3,0,3,1,1,21.01
3,3,0,2,1,1,23.68
4,3,1,4,1,1,24.59
...,...,...,...,...,...,...
239,2,0,3,1,1,29.03
240,2,1,2,0,1,27.18
241,2,0,2,0,1,22.67
242,2,0,2,1,1,17.82


In [11]:
### Train Test split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,train_size=0.75,random_state=1)

In [12]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators=200).fit(X_train,y_train)

#### Lets Understand Our Model With Shapash 
In this section, we use the SmartExplainer Object from shapash.

- It allows users to understand how the model works with the specified data.
- This object must be used only for data mining step. Shapash provides another object for deployment.


In [13]:
# !pip install shapash

In [14]:
from shapash.explainer.smart_explainer import SmartExplainer

In [15]:
xpl = SmartExplainer()

In [16]:
xpl.compile(
    x=X_test,
    model=regressor,
   
)

Backend: Shap TreeExplainer


In [17]:
xpl

#### Lets Understand the results of your trained model
Then, we can easily get a first summary of the explanation of the model results.

- Here, we chose to get the 3 most contributive features for each prediction.
- We used a wording to get features names more understandable in operationnal case.

In [18]:
app = xpl.run_app(title_story='Tips Dataset')

INFO:numexpr.utils:NumExpr defaulting to 4 threads.


Dash is running on http://0.0.0.0:8050/



/home/rakesh/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:1490: FutureWarning:

Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.

INFO:shapash.webapp.smart_app:Dash is running on http://0.0.0.0:8050/



 * Serving Flask app "shapash.webapp.smart_app" (lazy loading)


INFO:root:Your Shapash application run on http://kmaster:8050/


 * Environment: production


INFO:root:Use the method .kill() to down your app.


   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://0.0.0.0:8050/ (Press CTRL+C to quit)


In [19]:
predictor = xpl.to_smartpredictor()

In [20]:
predictor.save('./predictor.pkl')

In [21]:
from shapash.utils.load_smartpredictor import load_smartpredictor
predictor_load = load_smartpredictor('./predictor.pkl')

#### Make a prediction with your SmartPredictor
In order to make new predictions and summarize local explainability of your model on new datasets, you can use the method add_input of the SmartPredictor.

- The add_input method is the first step to add a dataset for prediction and explainability.
- It checks the structure of the dataset, the prediction and the contribution if specified.
- It applies the preprocessing specified in the initialisation and reorder the features with the order used by the model. (see the documentation of this method)
- In API mode, this method can handle dictionnaries data which can be received from a GET or a POST request.
- Add data
- The x input in add_input method doesn't have to be encoded, add_input applies preprocessing.

In [22]:
predictor_load.add_input(x=X, ypred=y)

In [23]:
detailed_contributions = predictor_load.detail_contributions()


In [24]:
detailed_contributions.head()

,tip,day,sex,size,smoker,time,total_bill
0,1.01,0.054369,0.108358,-0.012973,-0.029831,-0.009347,0.143741
1,1.66,0.118678,-0.050567,-0.006208,-0.085406,-0.023852,-1.219979
2,3.50,0.077601,-0.017984,-0.016762,-0.004774,0.001912,0.444623
3,3.31,0.132819,-0.018500,-0.014981,0.019339,-0.003551,0.050989
4,3.61,0.061703,0.057210,-0.061626,-0.008351,-0.018717,-0.229254


#### Summarize explanability of the predictions
- You can use the summarize method to summarize your local explainability
- This summary can be configured with modify_mask method so that you have explainability that meets your operational needs.

In [25]:

predictor_load.modify_mask(max_contrib=3)

In [26]:
explanation = predictor_load.summarize()

In [27]:
explanation.head()

,tip,feature_1,value_1,contribution_1,feature_2,value_2,contribution_2,feature_3,value_3,contribution_3
0,1.01,total_bill,16.99,0.143741,sex,1,0.108358,day,3,0.0543686
1,1.66,total_bill,10.34,-1.21998,day,3,0.118678,smoker,1,-0.0854061
2,3.50,total_bill,21.01,0.444623,day,3,0.0776007,sex,0,-0.0179839
3,3.31,day,3,0.132819,total_bill,23.68,0.0509895,smoker,1,0.0193392
4,3.61,total_bill,24.59,-0.229254,day,3,0.0617025,size,4,-0.0616264


INFO:werkzeug:127.0.0.1 - - [08/Apr/2021 13:08:27] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [08/Apr/2021 13:08:27] "GET /_dash-component-suites/dash_renderer/react@16.v1_8_3m1617867292.14.0.min.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [08/Apr/2021 13:08:27] "GET /assets/style.css?m=1617867293.759956 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [08/Apr/2021 13:08:27] "GET /_dash-component-suites/dash_renderer/react-dom@16.v1_8_3m1617867292.14.0.min.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [08/Apr/2021 13:08:27] "GET /_dash-component-suites/dash_renderer/polyfill@7.v1_8_3m1617867292.8.7.min.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [08/Apr/2021 13:08:27] "GET /assets/material-icons.css?m=1617867293.759956 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [08/Apr/2021 13:08:27] "GET /_dash-component-suites/dash_renderer/prop-types@15.v1_8_3m1617867292.7.2.min.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [08/Apr/2021 13:08:27] "GET /_dash-component-suites/dash_bootstr

INFO:werkzeug:192.168.1.5 - - [08/Apr/2021 13:09:25] "GET /_dash-layout HTTP/1.1" 200 -
INFO:werkzeug:192.168.1.5 - - [08/Apr/2021 13:09:25] "GET /assets/favicon.ico?m=1617867293.7559557 HTTP/1.1" 200 -
INFO:werkzeug:192.168.1.5 - - [08/Apr/2021 13:09:25] "GET /assets/shapash-fond-fonce.png HTTP/1.1" 200 -
INFO:werkzeug:192.168.1.5 - - [08/Apr/2021 13:09:25] "GET /assets/settings.png HTTP/1.1" 200 -
INFO:werkzeug:192.168.1.5 - - [08/Apr/2021 13:09:25] "GET /_dash-component-suites/dash_core_components/async-dropdown.v1_13_0m1604001894.js HTTP/1.1" 200 -
INFO:werkzeug:192.168.1.5 - - [08/Apr/2021 13:09:25] "GET /_dash-component-suites/dash_core_components/async-graph.v1_13_0m1604001894.js HTTP/1.1" 200 -
INFO:werkzeug:192.168.1.5 - - [08/Apr/2021 13:09:25] "GET /assets/reload.png HTTP/1.1" 200 -
INFO:werkzeug:192.168.1.5 - - [08/Apr/2021 13:09:25] "GET /_dash-component-suites/dash_table/async-table.v4_11_0m1604001235.js HTTP/1.1" 200 -
INFO:werkzeug:192.168.1.5 - - [08/Apr/2021 13:09:25]

INFO:werkzeug:127.0.0.1 - - [08/Apr/2021 13:10:42] "POST /_dash-update-component HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [08/Apr/2021 13:10:42] "POST /_dash-update-component HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [08/Apr/2021 13:10:44] "POST /_dash-update-component HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [08/Apr/2021 13:10:44] "POST /_dash-update-component HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [08/Apr/2021 13:10:49] "POST /_dash-update-component HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [08/Apr/2021 13:10:49] "POST /_dash-update-component HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [08/Apr/2021 13:10:50] "POST /_dash-update-component HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [08/Apr/2021 13:10:50] "POST /_dash-update-component HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [08/Apr/2021 13:10:52] "POST /_dash-update-component HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [08/Apr/2021 13:10:53] "POST /_dash-update-component HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [0